# NoteChat 使用简介

[Github Repo & English Introduction](https://github.com/firezym/jupyterlab-notechat) || [点击查看介绍视频](https://github.com/user-attachments/assets/b4b8a840-e5e9-46ae-ac5f-4fd218e253b3)

![intro_how_to_use.gif](https://raw.githubusercontent.com/firezym/jupyterlab-notechat/main/images/intro_how_to_use.gif)


# 插件安装

```bash
# 安装NoteChat扩展
pip install jupyterlab-notechat

# 在扩展设置或环境变量中定义LLM密钥
# Linux
export OPENAI_API_KEY=your_key
# Windows
$env:OPENAI_API_KEY = "your_key"
```

# 功能亮点

**Notechat类似于`Github Copilot`、`Cursor`或`jupyterlab-ai`，但不具备快速代码生成能力，更像一个可以在jupyterlab环境下和notebook交互的聊天机器人。**

**NoteChat工具的初衷是让用户能够进行`更加精准和可控的对话`，包括但不限于：**

- **尽可能将所有notebook元素纳入对话上下文，例如单元格执行输出、代码和 markdown 输入、外部文件等**
- **通过单元格的唯一ID、相对ID或范围ID精确指定单元格或文件引用**
- **Python内核可以直接和笔记本单元格代码、单元格输出以及AI生成的文本进行交互或引用**
- **按序执行Python代码和AI生成的文本**
- 手动操作 AI 生成的文本作为上下文
- ...

**帮助用户更好地利用 LLM 的优势，包括：**

- 用notebook cell进行常规对话流（尽管用户体验不如对话流畅）
- 撰写结构化文章或长报告
- 使用 LLM 模型进行数据和文本分析
- 辅助编程
- 将 LLM 知识引入并存储为笔记本
- ...

![基本界面](https://raw.githubusercontent.com/firezym/jupyterlab-notechat/main/images/intro_menu.bmp)


# **创建用户消息**

直接新建一个单元格输入问题，或者使用 `Add a User Chat Cell` 命令，或者点击对应菜单或单元格按钮 ![单元格按钮](https://raw.githubusercontent.com/firezym/jupyterlab-notechat/main/images/intro_add_message.bmp) 添加一个以 **user** 开头的 markdown 单元格，更清楚地表明这是用户创建的对话消息。


# **行内参数设置**

在单元格的第一行以 `@parameter xxx` 的形式添加自定义参数设置，但不要使用换行符。主要参数和示例如下：

- **`@refs`** 指定当前单元格和 AI 对话所引用的其他单元格，使上下文更加精确。例如在撰写长报告时，每个章节只需要看到初始提纲，无需看到其他章节内容。以下示例中，除非指定范围引用和“alone”模式，引用通常是叠加的。例如：
  - `@refs -8, +2, 2 be73e0fc-6e1c-4d49-a288-94e3f7ec8215`：将引用当前消息前第 8 个单元格、后第 2 个单元格、第 2 个单元格，以及唯一 ID 为 "be...15" 的单元格。分隔符可以是 `, | ; ;`。如果指定 ID 不包含范围引用（如冒号 : 或波浪号 ~），还会添加设置中默认的上下文范围。
  - `@refs alone -8, +2`：如果指定了 `alone`、`single` 或 `sole` 等词，表示忽略默认的额外范围引用，仅使用当前单元格指定的单一或范围引用。如果只有 "alone"，表示仅查看当前单元格。
  - `@refs -10:0`：引用当前消息前 10 个单元格的内容。范围可以加 `[]` 提高可读性，例如 `@refs [-10:0]`。
  - `@refs :+2 'xxxxxx'`：引用从文档开头到当前单元格下第 2 个单元格的所有内容，以及唯一 ID 为 'xxxxxx' 的单元格。
  - `@refs 2:8`：如果范围中没有 + 或 -，则表示文档中第 2 至第 8 个单元格的绝对 ID 顺序。
- **`@files file_path/file_name`** 可跨文件进行全文引用。请避免使用容易引起混淆的字符（如空格）。路径中可以包含特殊处理的 `@` 字符。目前支持的文件类型包括：文本类型（txt/md/py/js/ts/sh/bat/json/xml/log/config/ini/yaml/yml），表格类型（csv/xlsx/xls），文档类型（pdf/docx/pptx/html/htm）。除 ipynb 文件可包含图片外，其他文件解析尚不支持图片。
- **`@num_prev_cells 8`** 在当前单元格对话中，覆盖系统默认的向前引用范围为 8。
- **`@prompt xyz`** 使用 xyz 替换系统默认提示；请勿使用换行符。
- **`@add_prompt xyz`** 将 xyz 添加到系统默认提示中以形成新的提示。
- **`@model o1-preview`** 指定 LLM 模型为 gpt-4o-mini，便于以低成本测试初始想法。请查阅 OpenAI 官网了解模型信息。默认是 `@model gpt-4o`，也支持 `@model moonshot-v1-32k`（支持 8k、128k）。
- **`@use_vison false`** 不使用图像视觉模型；默认值为 true。可直接在 markdown 单元格中粘贴截图或图片，但目前不支持图片 URL。
- **`@max_input/@max_output 888`** 设置输入和输出的最大 token 数量。如果超过此值，将按绝对 ID 原顺序截断，但会优先保留提示和当前单元格。图像 token 计数尚不支持。
- **`@temperature 0.5`** 设置 LLM 模型的随机性，范围为 0~1。
- **`@timeout 600`** 设置模型的最大响应时间。
- **`单notebook级别参数设置`** 为了确保可重现性，可在 Notebook 元数据中通过属性检查器（笔记本右上角齿轮图标）下的 ADVANCED TOOLS 添加 `"notechat":{"param":value}` 来覆盖参数，例如设置笔记本级的提示。注意，此处的 param 不需要添加 @。覆盖优先级为用户 > 助手 > 笔记本 > 设置。
  ![参数设置](https://raw.githubusercontent.com/firezym/jupyterlab-notechat/main/images/intro_notebook_level_params.bmp)


# **顺序执行**

支持从上到下按顺序运行 Python 代码单元格和 LLM 支持的用户与助手对话流：方便长流程工作，如通过数据和 LLM 汇总分析自动更新报告。

![顺序执行](https://raw.githubusercontent.com/firezym/jupyterlab-notechat/main/images/intro_run_sequential.bmp)

# **Python 内核与 LLM 消息的交互**

Markdown 单元格和代码单元格的源文本可以直接以 `_refs["unique id"]` 的形式在当前内核程序中引用，方便用户在 Python 内核和 LLM 消息之间进行文本输入输出交互。

# **信息、帮助按钮和命令**

以字符串 xxxxxx 的形式获取当前单元格的唯一 ID 和从 0 开始计数的绝对 ID、当前单元格想要引用的 ID、@param 个性化参数以及其他信息。单击时，当前单元格的唯一 ID 引用将复制到剪贴板供用户参考。跨笔记本使用时，请直接使用 Python 程序读取 .ipynb 文件作为 JSON 数据以查找对应唯一 ID 的单元格信息。

![显示引用1](https://raw.githubusercontent.com/firezym/jupyterlab-notechat/main/images/intro_show_refs.bmp)

![显示引用2](https://raw.githubusercontent.com/firezym/jupyterlab-notechat/main/images/intro_show_refs_result.bmp)

# **跨notebook引用cell**

首先在外部notebook中，使用快捷键`ALT+Q`或工具栏上的`Show & Copy Cell ID for Ref`获取目标cell ID，然后用json组件load外部notebook，读取目标cell相关内容


In [2]:
import json
# 目标notebook文件路径
notebook_path = 'README-Example.ipynb'
# 目标单元格的ID
target_cell_id = '7ee61417-0c9b-495d-9015-ad17c5320358'
# 加载笔记本文件
with open(notebook_path, 'r', encoding='utf-8') as notebook_file:
    notebook_content = json.load(notebook_file)
# 初始化变量来存储结果
target_source, target_outputs = None, None
# 遍历笔记本中的所有单元格
for cell in notebook_content['cells']:
    # 检查单元格ID是否匹配
    if cell.get('id') == target_cell_id:
        # 获取source
        target_source = cell.get('source', [])
        # 获取outputs
        target_outputs = cell.get('outputs', [])
        break
# 检查是否找到目标单元格，并打印结果
if target_source is not None and target_outputs is not None:
    print("找到了目标单元格！")
    print("Source:")
    print('\n'.join(target_source))
    print("Outputs:")
    print(target_outputs)
else:
    print("未找到具有指定ID的单元格。")

找到了目标单元格！
Source:
**assistant**



La suite de Fibonacci est une séquence de nombres entiers dans laquelle chaque nombre est la somme des deux précédents, à partir de 0 et 1. Elle commence généralement par les nombres 0 et 1. La suite s'exprime donc mathématiquement ainsi :



\[ F(0) = 0 \]

\[ F(1) = 1 \]

\[ F(n) = F(n-1) + F(n-2) \quad \text{pour } n \geq 2 \]



Les premiers termes de la suite de Fibonacci sont : 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, etc.



Cette suite apparaît dans de nombreux contextes en mathématiques et en sciences naturelles, notamment dans les modèles de croissance des populations, les structures végétales, et même dans les arts visuels en raison de ses propriétés esthétiques. Une des propriétés intéressantes de la suite de Fibonacci est sa relation avec le nombre d'or (phi, \(\varphi\)), qui est approximativement égal à 1,618. À mesure que vous progressez dans la suite, le rapport entre deux nombres consécutifs tend vers ce nombre d'or.



Si vous souhaitez, je

# **表格识别**

目前没有很好的工具来处理 HTML，建议使用 pandas 处理数据，并尝试通过 `df.to_markdown()` 将其转换为 markdown 表格格式，以便 LLM 更好地识别和处理。


# 常见示例

## 整理数据

**可以从pdf中把一些复制粘贴较为费事的数据拷贝出来，制作成表格**


招商银行公告

2023Q3：
报告期内，本公司新生成不良贷款454.95亿元，同比减少6.74亿元；不良贷款生成率（年化） 1.03%，
同比下降0.10个百分点。其中，公司贷款不良生成额92.08亿元，同比减少42.74亿元，主要是房地产业
贷款不良生成额同比减少；零售贷款（不含信用卡）不良生成额65.69亿元，同比增加9.56亿元，信用
卡新生成不良贷款297.18亿元，同比增加26.44亿元，零售贷款不良生成同比增加， 主要是2022年末经
济运行疲弱带来的滞后影响，整体在可控范围内。
本公司继续积极处置不良资产，报告期内共处置不良贷款429.62亿元，其中，常规核销168.80亿元，
清收82.79亿元，不良资产证券化177.13亿元，通过抵债、转让、重组上迁、减免等其他方式处置0.90
亿元。
本公司坚持审慎、稳健的拨备计提政策，按照金融工具准则要求，以预期信用损失模型为基础，基于
客户的违约概率、违约损失率等风险量化参数，结合宏观前瞻性的调整，充分计提信用风险损失准备，
并不断夯实重点领域风险抵补能力。截至报告期末，本公司贷款损失准备余额2,672.50亿元，较上年
末增加138.37亿元；拨备覆盖率468.96%，较上年末上升1.53个百分点；贷款拨备率4.39%，较上年末
下降0.04个百分点。 2023年1-9月，本公司信用成本（年化） 0.90%，同比上升0.09个百分点。

2023H1:
报告期内，本公司新生成不良贷款305.09亿元，同比减少1.93亿元 ；不良贷款生成率（年化） 1.04%，同比下降
0.09个百分点。从业务大类看，公司贷款不良生成额56.93亿元，同比减少33.80亿元 ；零售贷款（不含信用卡）
不良生成额43.69亿元，同比增加7.88亿元 ；信用卡新生成不良贷款204.47亿元，同比增加23.99亿元。从地
区看，不良生成主要分布在长三角、环渤海地区及总行（信用卡贷款）；从行业看，不良生成主要集中在房地产
业 ；从客群看，不良生成多数为国标中型企业。
本公司持续夯实预期信用损失法实施基础，并始终坚持价值客户选择，优化资产组合配置，风险抵补充足，具备
较强的风险抵御能力。截至报告期末，本公司贷款损失准备余额2,636.18亿元，较上年末增加102.05亿元 ；拨
备覆盖率465.73%，较上年末下降1.70个百分点 ；贷款拨备率4.41%，较上年末下降0.02个百分点 ；报告期信
用成本（年化） 0.90%，同比上升0.07个百分点。
报告期内，本公司积极处置不良资产，运用多种途径化解风险资产，报告期内共处置不良贷款282.78亿元，其
中，常规核销110.51亿元，不良资产证券化113.76亿元，清收56.37亿元，通过抵债、转让、重组上迁、减免
等其他方式处置2.14亿元。

2023Q1
报告期内，本公司新生成不良贷款160.23亿元，同比增加5.87亿元；不良贷款生成率（年化） 1.09%，同比
下降0.07个百分点。其中，公司贷款不良生成额30.70亿元， 同比减少21.63亿元，主要是房地产业贷款不良
生成节奏同比有所放缓；零售贷款（不含信用卡）不良生成额24.67亿元，同比增加10.32亿元，信用卡新生
成不良贷款104.86亿元，同比增加17.18亿元，零售贷款不良生成同比增加，主要是2022年末经济表现相对
疲弱带来的滞后影响， 整体可控。
本公司继续积极处置不良资产，报告期内共处置不良贷款136.63亿元，其中，常规核销44.58亿元，清收28.02
亿元，不良资产证券化60.96亿元，通过抵债、转让、重组上迁、减免等其他方式处置3.07亿元。
本公司坚持审慎、稳健的拨备计提政策，按照金融工具准则要求，以预期信用损失模型为基础，基于客户
的违约概率、违约损失率等风险量化参数，结合宏观前瞻性的调整，充分计提信用风险损失准备，并不断
夯实重点领域风险抵补能力。截至报告期末，本公司贷款损失准备余额2,620.27亿元，较上年末增加86.14
亿元；拨备覆盖率463.19%，较上年末下降4.24个百分点；贷款拨备率4.39%，较上年末下降0.04个百分点。
2023年1-3月，本公司信用成本（年化） 1.04%，同比上升0.23个百分点， 主要是一季度贷款规模增加导致
贷款拨备支出增加。

**user** @refs -1:0

帮我把上面的文本中的关于资产质量的各项披露数据整理成表格，横向是报告期从小到大，纵向是各项数据名称，导出为excel

**assistant**

为了整理招商银行公告中关于资产质量的各项披露数据并导出为Excel表格，我们可以使用Python的`pandas`库来处理数据。以下是一个示例代码，可以帮助实现这一目标：

```python
import pandas as pd

# 定义报告期
report_periods = ['2023Q1', '2023H1', '2023Q3']

# 定义各项数据
data = {
    '新生成不良贷款（亿元）': [160.23, 305.09, 454.95],
    '不良贷款生成率（年化）': [1.09, 1.04, 1.03],
    '公司贷款不良生成额（亿元）': [30.70, 56.93, 92.08],
    '零售贷款不良生成额（亿元）': [24.67, 43.69, 65.69],
    '信用卡新生成不良贷款（亿元）': [104.86, 204.47, 297.18],
    '贷款损失准备余额（亿元）': [2620.27, 2636.18, 2672.50],
    '拨备覆盖率（%）': [463.19, 465.73, 468.96],
    '贷款拨备率（%）': [4.39, 4.41, 4.39],
    '信用成本（年化）': [1.04, 0.90, 0.90],
    '处置不良贷款（亿元）': [136.63, 282.78, 429.62],
    '常规核销（亿元）': [44.58, 110.51, 168.80],
    '清收（亿元）': [28.02, 56.37, 82.79],
    '不良资产证券化（亿元）': [60.96, 113.76, 177.13],
    '其他方式处置（亿元）': [3.07, 2.14, 0.90]
}

# 创建DataFrame
df = pd.DataFrame(data, index=report_periods)

# 导出为Excel
df.to_excel('资产质量数据.xlsx')

print("数据已成功导出为 '资产质量数据.xlsx'")
```

这段代码先定义了报告期和相应的资产质量数据，然后使用`pandas`库将数据存储在一个DataFrame中，最后导出为Excel文件。运行这段代码将生成一个名为`资产质量数据.xlsx`的文件，其中包含了整理后的资产质量数据。

<div style="text-align: right; color: lightgray; font-style: italic; font-size: x-small;">_ref || _refs["4995d433-cab7-4559-a69b-e24ef2a20f78"]</div>

## 代码Copilot

In [45]:
bb

NameError: name 'bb' is not defined

**assistant**

看来您在代码单元格中输入了`bb`，这导致了一个`NameError`，因为`bb`在当前的代码环境中没有被定义。

如果`bb`是一个变量或函数名称，请确保它在之前的代码中已被正确定义。例如：

```python
# 假设 bb 是一个变量
bb = 42
print(bb)
```

或者，如果`bb`是一个函数，确保函数已被定义：

```python
def bb():
    return "Hello, world!"

print(bb())
```

请根据您想要实现的功能，确保定义了`bb`，然后再尝试运行。如果有其他问题或需要进一步的帮助，请随时告诉我！

<div style="text-align: right; color: lightgray; font-style: italic; font-size: x-small;">_ref || _refs["8560c6e2-eb89-4591-bb31-e15f2c43b330"]</div>

# NoteChat工作环境安装

- **使用miniconda或conda来管理隔离的环境**，以免和其他代码环境冲突，这里以dev环境命名为例，可以改成任何你所需要的名字

- **移除dev环境**，确保dev命名的环境卸载干净：

  `conda env remove -n dev`
  
- **创建新的dev环境**，使用最新的jupyterlab：

  `conda create -n dev --override-channels --strict-channel-priority -c conda-forge -c nodefaults --yes python=3.11 jupyterlab=4.3.1 nodejs=18`
  
- **激活环境**，并安装依赖包和你平时常用的工作环境包，这里plotly和jupyterlab-gridwidth不是必须

  `conda activate dev`
  
  `pip install pandas tabulate jupyterlab-gridwidth`

  `pip install plotly colour seaborn kaleido opencv-python` (可选)
  
- **cd到安装包文件夹并安装NoteChat**，假设NoteChat插件包在D盘xxxxxx文件夹下，这里是Windows演示，Linux同理

  `cd D:\xxxxxx`
  
  `pip install jupyterlab-notechat`
  
- 可以检查插件是否安装成功

  `jupyter labextension list`
  
  `jupyter server extension list`

- **设定OPENAI_API_KEY**，两种方式：

  **1. 一种是设定在系统变量中**
  
  - **Linux**中，在当前运行的命令行session中输入指令，可以存好每次启动的系列命令，因为这个key只在当前session适用
  
    `export OPENAI_API_KEY=your_key`
  
    `echo $OPENAI_API_KEY`
  
  - **Windows**中，可以在`我的电脑->属性->高级系统设置->环境变量->系统变量`，新建一个`OPENAI_API_KEY`的变量，值设为`your_key`，设定好后长期适用，但需要重启一下当前运行的命令行terminal，这样这个session才能获得最新的key，或在当前powershell的session使用

    `$env:OPENAI_API_KEY = "your_key"`
  
    `echo $env:OPENAI_API_KEY`

  **2. 一种是在jupyterlab启动后**，在`菜单->Settings->Settings Editor->NoteChat`，在页面中更改`@openai_api_key`的`None`值为`your_key`

  - 这个页面还可以设置NoteChat其他变量，比如，对话流中`@num_prev_cells`为当前Cell向上最多可以看几个Cell，这里默认值为20
  
- **激活环境并定位到工作文件夹目录，启动jupyterlab**，这里以`D:\xxxxxx\dev`为例：

    `conda activate dev`
  
    `cd D:\xxxxxx\dev`
  
    `jupyter lab`
  
- 如果想指定的目标文件夹和端口，使用如下指令，建议也要指定`workspaces_dir`和`user_settings_dir`，这样建立多个独立jupyterlab服务的个性化设定不会冲突

    `conda activate dev`
  
    `jupyter lab --notebook-dir=D:\xxxxxx\dev --LabServerApp.workspaces_dir=D:\xxxxxx\path\dev\workspaces --LabServerApp.user_settings_dir=D:\xxxxxx\path\dev\user-settings --port=8889 --no-browser --ip 0.0.0.0`

- **启动notebook实例**，可以看到菜单栏和notebook工作栏有NoteChat的菜单或按钮，请查看菜单栏帮助`Alt+H`、`###使用说明README###.ipynb`和几个notebook example，有助于理解使用场景和运行机制，如果你想当成一个类似于openai的chatgpt来用，目前没有集成RAG模块，建议增加向上回溯的Cell数量`菜单->Settings->Settings Editor->NoteChat->Number of Cells Before the Active Cell As Context for AI Assistant`，比如设定为10或更多，因为新版的GPT能看很长的上下文，这样保证看到较多的单元格，但正如前言中所说，Notechat工具初衷是让用户有更精准可控的对话，善用单元格附加参数，将给报告自动化撰写、数据分析研究带来更为精准的提升：）
